<a href="https://colab.research.google.com/github/kochhar/keras-word2vec/blob/master/Keras_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kochhar/keras-word2vec

Cloning into 'keras-word2vec'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 45 (delta 17), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [2]:
import os
os.chdir('/content/keras-word2vec')
os.getcwd()

'/content/keras-word2vec'

In [35]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
Unpacking objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
From https://github.com/kochhar/keras-word2vec
   50e51cf..13fe8f4  master     -> origin/master
Updating 50e51cf..13fe8f4
Fast-forward
 word2vec/__init__.py | 12 ++++--------
 1 file changed, 4 insertions(+), 8 deletions(-)


In [36]:
import collections
import logging
import numpy as np
from sklearn.model_selection import train_test_split
import word2vec as w2v
from importlib import reload
w2v = reload(w2v)

logging.basicConfig(format="[%(asctime)s]%(levelname)s:%(name)s:%(message)s",
                    level=logging.INFO)
logging.info('logging configured')


[2018-10-30 03:26:30,161]INFO:root:logging configured


In [15]:
filename = 'data/text8.zip'
logging.info('Reading corpus')
training_corpus = w2v.str_from_zipfile(filename).split()
logging.info('Corpus: %s words', len(training_corpus))

[2018-10-30 03:07:14,666]INFO:root:Reading corpus
[2018-10-30 03:07:16,853]INFO:root:Corpus: 17005207 words


In [16]:
logging.info('Creating vocabulary')
counts = [('UNK', -1)] + collections.Counter(training_corpus).most_common(int(1e4-1))
vocab = w2v.Vocabulary([word for (word, _) in counts ])
logging.info('Encoding corpus')
tokens = vocab.encode(training_corpus)
logging.info("Vocab: %s tokens, Sample: %s", vocab.size, list(zip(training_corpus[:7], tokens[:7])))
del training_corpus


[2018-10-30 03:07:20,751]INFO:root:Creating vocabulary
[2018-10-30 03:07:24,523]INFO:root:Encoding corpus
[2018-10-30 03:07:27,941]INFO:root:Vocab: 10000 tokens, Sample: [('anarchism', 5234), ('originated', 3081), ('as', 12), ('a', 6), ('term', 195), ('of', 2), ('abuse', 3134)]


In [18]:
logging.info('Creating skipgrams')
word_target, word_context, labels = sgns.skipgrams(tokens, window_size=3)
logging.info('Training skipgrams: %s, Sample: %s', len(word_target), 
             list(zip(word_target[:10], word_context[:10], labels[:10])))
del tokens

[2018-10-30 03:07:38,411]INFO:root:Creating skipgrams
[2018-10-30 03:12:05,531]INFO:root:Training skipgrams: 30022202, Sample: [(9817, 3, 1), (1460, 1264, 0), (6786, 831, 0), (6777, 7884, 0), (278, 990, 0), (606, 6433, 0), (5230, 8627, 0), (669, 9626, 0), (1057, 1614, 0), (2831, 4311, 0)]


In [0]:
class Similarity(object):
    def __init__(self, sim_model):
        """Params:
            - sim_model: A keras model which outputs a similarity score given
              two vectors as inputs"""
        self.sim_model = sim_model

    def most_similar(self, examples, vocab, top_k=10):
        sim_fn = self.sim
        contexts = range(vocab.size)
        # compute the similarity between the example and every context word in the vocabulary
        sims = [(ex, sim_fn(ex, contexts)) for ex in examples]
        sims = [(ex, (-sim).argsort()[1:top_k+1], sim) for (ex, sim) in sims]
        return [(ex, top_idx, sim[top_idx]) for (ex, top_idx, sim) in sims]

    def sim(self, target, contexts):
        context_b = np.array(contexts)
        # the target word for every item in the batch is the same
        target_b = np.array([target]*len(contexts))
        sim_b = self.sim_model.predict_on_batch([target_b, context_b])
        
        # reshape from (len(contexts), 1) to (len(contexts),)
        sim_b = sim_b.reshape(context_b.shape)
        return sim_b


In [0]:
# Build the model
sgns = w2v.SkipGramNegSample(vocab_size=vocab.size, embedding_dim=300)
model, validation_model = sgns.model()
sim_eval = Similarity(validation_model)

# Validation set for inspecting training progress
# Max index of the random sampling from tokens (implies choosing
# from the the top-N occurring tokens)
max_token_idx = 128
valid_samples = np.random.choice(np.arange(24, max_token_idx), size=16, replace=False)

# Feeding to the network
batch_size, offset = 128, 127
tgt_b, ctx_b, lbl_b = np.zeros((batch_size,)), np.zeros((batch_size,)), np.zeros((batch_size,))
# training loop over 1e6 epochs with batches of size 32
for epoch in range(5):
  for idx in range(0, len(word_target), batch_size):
      tgt_b[0:offset,] = word_target[idx:idx+offset]
      ctx_b[0:offset,] = word_context[cnt:cnt+offset]
      lbl_b[0:offset,] = labels[cnt:cnt+offset]
      loss = model.train_on_batch([tgt_b, ctx_b], lbl_b)

      if idx % 51200 == 0:
          logging.info("Epoch: %s, Iteration %s, loss=%s", epoch, idx, loss)
      if idx % 1024000 == 0:
          most_similar = sim_eval.most_similar(valid_samples, vocab)
          log_str = '\n'+'\n'.join(['Nearest to {}: {}, Median sim: {:.3}'
                                       .format(vocab.decode([ex]), ', '.join(vocab.decode(top_idx)), scores[4])
                                   for (ex, top_idx, scores) in most_similar])
          logging.info(log_str)


[2018-10-30 04:49:06,875]INFO:root:Epoch: 0, Iteration 0, loss=0.6931793
[2018-10-30 04:49:07,258]INFO:root:
Nearest to ['english']: bandwidth, dinner, decorated, unclear, cool, concerned, instability, denominations, cambodia, center, Median sim: 0.195
Nearest to ['under']: peirce, approach, matters, potential, aware, classes, causing, mason, oct, dundee, Median sim: 0.186
Nearest to ['some']: seats, automatic, media, car, mickey, focused, joey, offence, persecution, magic, Median sim: 0.195
Nearest to ['were']: surrendered, program, suffix, branch, conferences, westminster, consumed, division, object, small, Median sim: 0.189
Nearest to ['united']: stomach, der, fred, carnegie, drops, pleasant, qquad, par, revision, end, Median sim: 0.183
Nearest to ['e']: capital, penis, programme, established, tribune, emissions, sculpture, goddess, champions, vowel, Median sim: 0.184
Nearest to ['be']: torah, folk, touch, assert, arrives, begin, programmes, richard, vehicles, fl, Median sim: 0.201
